In [1]:
import pandas as pd

In [2]:
import numpy as np

In [6]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [7]:
df.cat =df.select_dtypes(include=[object]) 

C:\Users\pkitt\AppData\Local\Temp\ipykernel_24964\1653857449.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [8]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_24964\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [9]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,Clayey,Sugarcane,28-28
1,Sandy,Millets,28-28
2,Sandy,Millets,17-17-17
3,Sandy,Barley,10-26-26
4,Red,Paddy,DAP


In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
le = LabelEncoder()

In [12]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [13]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,1,Sugarcane,28-28
1,4,Millets,28-28
2,4,Millets,17-17-17
3,4,Barley,10-26-26
4,3,Paddy,DAP


In [14]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [15]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,1,8,28-28
1,4,4,28-28
2,4,4,17-17-17
3,4,0,10-26-26
4,3,6,DAP


In [16]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [17]:
df_train = pd.concat([df.num,df.cat],axis=1)

In [18]:
import tensorflow as tf

In [19]:
from tensorflow import keras

In [20]:
X = df_train[["Humidity","Moisture"]]

In [21]:
y = df_train[["Fertilizer Name"]]

In [22]:
pip install shap


Note: you may need to restart the kernel to use updated packages.


In [23]:
from tensorflow.keras.regularizers import l2

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(128, activation='relu', kernel_regularizer=l2(0.00001)),
    Dense(64, activation='relu', input_shape=(4,)),
    Dense(32, activation='relu'),
    Dense(10, activation='softmax') # or 'sigmoid' for binary classification
])
model.compile(
    optimizer='sgd',  # or 'sgd', 'rmsprop', etc.
    loss='sparse_categorical_crossentropy',  # depends on your task
    metrics=['accuracy']  # optional, but helpful
)

C:\Users\pkitt\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
def ordered_classwise_batches(df, feature_cols, label_col, batch_size=32):
    batches = []
    class_labels = sorted(df[label_col].unique())  # Ensures order: 0, 1, 2, ...

    for label in class_labels:
        group = df[df[label_col] == label].reset_index(drop=True)
        for i in range(0, len(group), batch_size):
            batch = group.iloc[i:i + batch_size]
            X_batch = batch[feature_cols].values
            y_batch = batch[label_col].values
            batches.append((X_batch, y_batch))

    return batches

In [35]:
feature_cols = ['Nitrogen','Potassium','Moisture','Temparature','Humidity','Soil Type']
label_col = 'Fertilizer Name'

batches = ordered_classwise_batches(df_train, feature_cols, label_col, batch_size=64)

# Confirm that first few batches are in order
for i, (X, y) in enumerate(batches[:10]):
    print(f"Batch {i} - Fertilizer Label(s): {np.unique(y)}")

Batch 0 - Fertilizer Label(s): [0]
Batch 1 - Fertilizer Label(s): [0]
Batch 2 - Fertilizer Label(s): [0]
Batch 3 - Fertilizer Label(s): [0]
Batch 4 - Fertilizer Label(s): [0]
Batch 5 - Fertilizer Label(s): [0]
Batch 6 - Fertilizer Label(s): [0]
Batch 7 - Fertilizer Label(s): [0]
Batch 8 - Fertilizer Label(s): [0]
Batch 9 - Fertilizer Label(s): [0]


In [36]:
for i, (X_batch, y_batch) in enumerate(batches):
    X_batch = X_batch.astype('float32')
    metrics = model.train_on_batch(X_batch, y_batch)
    print(f"Batch {i+1}: Loss = {metrics[0]:.4f}, Accuracy = {metrics[1]*100:.2f}%")

Batch 1: Loss = 10.9820, Accuracy = 0.00%
Batch 2: Loss = 5.4911, Accuracy = 50.00%
Batch 3: Loss = 3.6608, Accuracy = 66.67%
Batch 4: Loss = 2.7456, Accuracy = 75.00%
Batch 5: Loss = 2.1965, Accuracy = 80.00%
Batch 6: Loss = 1.8305, Accuracy = 83.33%
Batch 7: Loss = 1.5690, Accuracy = 85.71%
Batch 8: Loss = 1.3729, Accuracy = 87.50%
Batch 9: Loss = 1.2204, Accuracy = 88.89%
Batch 10: Loss = 1.0983, Accuracy = 90.00%
Batch 11: Loss = 0.9985, Accuracy = 90.91%
Batch 12: Loss = 0.9153, Accuracy = 91.67%
Batch 13: Loss = 0.8449, Accuracy = 92.31%
Batch 14: Loss = 0.7846, Accuracy = 92.86%
Batch 15: Loss = 0.7323, Accuracy = 93.33%
Batch 16: Loss = 0.6865, Accuracy = 93.75%
Batch 17: Loss = 0.6461, Accuracy = 94.12%
Batch 18: Loss = 0.6103, Accuracy = 94.44%
Batch 19: Loss = 0.5781, Accuracy = 94.74%
Batch 20: Loss = 0.5492, Accuracy = 95.00%
Batch 21: Loss = 0.5231, Accuracy = 95.24%
Batch 22: Loss = 0.4993, Accuracy = 95.45%
Batch 23: Loss = 0.4776, Accuracy = 95.65%
Batch 24: Loss = 0.4

In [37]:
model.fit(X,y, epochs=5)

Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.0000e+00 - loss: 10.1713 
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.0000e+00 - loss: 1.7794
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5417 - loss: 1.1369
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 1.0000 - loss: 0.3107
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 1.0000 - loss: 0.1037


In [38]:
model.evaluate(X, y)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 1.0000 - loss: 0.0598 


[0.05923274904489517, 1.0]

In [39]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
model.fit(X, y, class_weight=dict(enumerate(class_weights)))

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - accuracy: 1.0000 - loss: 0.0553


In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
for i, (X_batch, y_batch) in enumerate(batches):
    X_batch = X_batch.astype('float32')
    metrics = model.train_on_batch(X_batch, y_batch)
    print(f"Batch {i+1}: Loss = {metrics[0]:.4f}, Accuracy = {metrics[1]*100:.2f}%")

In [42]:
import numpy as np

# Combine all batches into full training arrays
X_train = np.vstack([X for X, _ in batches])
y_train = np.hstack([y for _, y in batches])

model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 49s 5ms/step - accuracy: 0.1882 - loss: 1.7735 - val_accuracy: 0.0000e+00 - val_loss: 6.4220
Epoch 2/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 45s 5ms/step - accuracy: 0.1892 - loss: 1.7446 - val_accuracy: 0.0000e+00 - val_loss: 6.7346
Epoch 3/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 45s 5ms/step - accuracy: 0.1898 - loss: 1.7439 - val_accuracy: 0.0000e+00 - val_loss: 6.7839
Epoch 4/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 46s 5ms/step - accuracy: 0.1897 - loss: 1.7441 - val_accuracy: 0.0000e+00 - val_loss: 6.7805
Epoch 5/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 82s 5ms/step - accuracy: 0.1905 - loss: 1.7427 - val_accuracy: 0.0000e+00 - val_loss: 6.7649


In [43]:
from sklearn.svm import SVR

In [ ]:
from sklearn.model_selection import GridSearchCV

def optimize_svr(X, y):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    grid = GridSearchCV(SVR(), param_grid, cv=5)
    grid.fit(X, y)
    return grid.best_estimator_

best_model = optimize_svr(X_train, y_train)